In [ ]:
import pandas as pd
df1=pd.read_csv("/content/train.csv")
df2=pd.read_csv("/content/test.csv")

# Combine using concat()
df_train = pd.concat([df1, df2])

In [ ]:
df_train.head()

,category,sub_category,crimeaditionalinfo
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,I had continue received random calls and abusi...
1,Online Financial Fraud,Fraud CallVishing,The above fraudster is continuously messaging ...
2,Online Gambling Betting,Online Gambling Betting,He is acting like a police and demanding for m...
3,Online and Social Media Related Crime,Online Job Fraud,In apna Job I have applied for job interview f...
4,Online Financial Fraud,Fraud CallVishing,I received a call from lady stating that she w...


In [ ]:
print("Dataset Shape:", df_train.shape)
print("\nColumns in the dataset:", df_train.columns.tolist())

Dataset Shape: (124915, 3)

Columns in the dataset: ['category', 'sub_category', 'crimeaditionalinfo']


### **Empty or whitespace values, missing values, duplicates, charecters length < 150**

In [ ]:
df_train_cleaned = df_train.copy()
df_train_cleaned.dropna(subset=['crimeaditionalinfo'] ,inplace=True)

In [ ]:
# 1. Empty or whitespace values
def strip_warn(row):
    try:
        x = row.strip()
        return x
    except:
        print(row)
        return ""


print(df_train_cleaned[df_train_cleaned['crimeaditionalinfo'].str.strip().str.len() == 0].shape)
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(strip_warn)
df_train_cleaned = df_train_cleaned[df_train_cleaned['crimeaditionalinfo'].str.strip().str.len() > 0]

(1582, 3)


In [ ]:
# 2. Missing values
print(df_train_cleaned.isnull().sum())

category                 0
sub_category          8758
crimeaditionalinfo       0
dtype: int64


In [ ]:
# df_train_cleaned=df_train_cleaned.dropna(subset=["crimeaditionalinfo"])

In [ ]:
# 3. Duplicates
print(df_train_cleaned.duplicated().sum())

11295


In [ ]:
df_train_cleaned=df_train_cleaned.drop_duplicates()

In [ ]:
print(df_train_cleaned['crimeaditionalinfo'].duplicated().sum())

643


In [ ]:
df_train_cleaned = df_train_cleaned.drop_duplicates(subset=['crimeaditionalinfo'])

In [ ]:
# 4. Character length check
count_less_than_150 = df_train_cleaned[df_train_cleaned['crimeaditionalinfo'].str.len() <= 150].shape[0]
print("Number of rows with less than 150 characters in the crimeaditionalinfo column:", count_less_than_150)

Number of rows with less than 150 characters in the crimeaditionalinfo column: 8237


In [ ]:
# Keep only rows where text length is > 150 characters
df_train_cleaned = df_train_cleaned[df_train_cleaned['crimeaditionalinfo'].str.len() > 150]

# Reset index
df_train_cleaned.reset_index(drop=True, inplace=True)

In [ ]:
df_train_cleaned.shape

(103130, 3)

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
df_train_cleaned.sample(10)

,category,sub_category,crimeaditionalinfo
91134,Online Financial Fraud,Internet Banking Related Fraud,caller informed that he recived a message like his uno account will be block so he need to update his pancard so he fill the details and enter the otp after that amount has been deducted from his account he lost rs
3634,Online Financial Fraud,DebitCredit Card FraudSim Swap Fraud,FRAUD THROUGH CREDIT CARD \r\nBANK STATE BANK OF INDIA \t\r\nCREDIT CARD NUMBER \r\nTOTAL AMOUNT \r\nPLEASE HOLD REVERSE TOTAL AMOUNT IN THE COMPLAINT ACCOUNT AND TAKE NECESSARY ACTION
52865,Cyber Attack/ Dependent Crimes,Hacking/Defacement,"The issue actually started when I got this email, which at first glance seemed like spam. I usually ignore such things, but for some reason, I opened it this time. Afterward, my phone started freezing, and it wasn’t normal. I thought it was a technical glitch, but now I’m not so sure. Looking back, maybe I should have paid more attention to the little details, but now it's too late. My accounts are compromised. The more I tried to fix it, the more problems came up. My work account was notifiction, and then I got locked out of everything. I don’t think I’ve ever felt this violated before. My privacy is completely shattered, and I can’t do anything to make it stop. Even my laptop isn’t working right anymore. It’s so stange how everything just fell apart after that one email."
51864,Online Financial Fraud,UPI Related Frauds,i got call from this number he said i am calling from SBI belur brach will update Your PAN Details I will touch the link it will go to SBI APP normal opening procedure and ask pan number i put that number OTP came And i put tha number debited my account sir kindly consider my compliant for finding my money
13373,Online Financial Fraud,UPI Related Frauds,my name is DR janani Anthony on dec i send money from gpay for this person for matte jewel as order taken from whatsapp name of group shri collection but still not get my product after that i asked him to refund still now refunded plz take some action against this thank you
14541,RapeGang Rape RGRSexually Abusive Content,NaN,SirMadam \r\n I recieved WhatsApp video call from unknown number daily In video call there is sexual content I am a student so I get disturbed video call comes from another number daily at time aprox PM It has been days for this incident Please do something
81676,Online Financial Fraud,UPI Related Frauds,My tiffin service work he give me order for person I wants advance payment than he told me to cash note given me transfer army check QR Code scann and credit rs first time rs nd time than he debit th in my account benifecery account name Mahesh Kumar Prajapati mob no
27923,Online and Social Media Related Crime,Online Job Fraud,I was searching for a job in naukri app At that time I did not know that fake companies are in naukri app I trusted naukri app On th January I received a message from two companies They asked me to pay initial amount for the security deposit I believed them and paid that amount I paid Rs for one company and Rs for another company After that I realised that these are fake companies so I wanted my money back I asked one of the company to return my money back they did not respond properly So I moved to a lawyer through online but they also did nothing I finally came here to get justice
37877,Online Financial Fraud,UPI Related Frauds,Waytenz looks a fraudulent ecommerce website offer customers a veriety of products we ordered a product shown in above url given as St attachment product has not been delivered and they donot even respond to emails and email mentioned in website does not existkindly look into it to avoid further transactions
25031,Online Gambling Betting,Online Gambling Betting,This site is claiming to produce money on purchase of movies After purchasing the movie amount is not recovering I had bought movie named RAES at rs I am earning everyday on site but this is not being credited into my bank


### **Reduce noise in data**

In [ ]:
! pip install contractions -q

In [ ]:
# Import required libraries
import re
import unicodedata
import contractions


# Fix encoding and handle accented characters
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: unicodedata.normalize('NFKD', x))
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: x.encode('ascii', 'ignore').decode('utf-8'))

# Expand contractions
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: contractions.fix(x))

# Remove noise but keep periods
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: x.lower())
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: re.sub(r'\S+@\S+', '', x))
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: re.sub(r'http\S+|www.\S+', '', x))
# Modified to keep periods while removing other punctuation
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: re.sub(r'[^\w\s\.]', '', x))
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: re.sub(r'\s+', ' ', x))
df_train_cleaned['crimeaditionalinfo'] = df_train_cleaned['crimeaditionalinfo'].apply(lambda x: x.strip())

In [ ]:
df_train_cleaned.sample(10)

,category,sub_category,crimeaditionalinfo
1575,Online Financial Fraud,Internet Banking Related Fraud,fraud through internet banking related fraud bank union bank of india including andhra bank and corporation bank account no total amount please hold reverse total amount in the complaint account and take necessary action
12409,Cyber Terrorism,Cyber Terrorism,i got spam calls and messages on my what is app they are demanding for my with me and my fiance they were called our family persons too can you please resolve this worse situation they called us with lots of cyber number
44191,Online Financial Fraud,Fraud CallVishing,hi my wife received the fraud call on her number and the person was saying that he is sending rupees he was acting as fathers friend then he sent the gpay link and pay option was coming in google pay the fraud call number is kindly take some action on this person and check the details of this number he might already have scammed so many other people
20223,Online Financial Fraud,UPI Related Frauds,fraud through upi fraud phone pe bank punjab national bank including oriental bank of commerce and united bank of india account number total amount please hold reverse total amount in the complaint account and take necessary action
60332,Online Financial Fraud,UPI Related Frauds,sir mera mobile ko kisi n heak karke meri sim card ko galat tarrke se paryog karke or n jane kis tarikese mera mobile m sms aana band ho gye or usne mera account se sara payment nikal liya h kisi n ford karke
21939,Any Other Cyber Crime,Other,an fir is registered against the accused in police station camp palwal haryana on no action has been taken so far police has not taken any statement from complainent police is in collusion with accused and trying to save him accused is my husband and tapped my phones and emails accused tapped my phone various times and knew of my exact location he also hacked my emails and presented them in a court case telling exact pnr details of my train tickets he has been spying on me and it is a grave threat to my safety accused illegally obtained my birth certificate from my maternal place municipal corporation without my written consent he also presented my fake email prints regarding me visiting some dating sites how he could got hold of my phone and emails he is trying to harass me by doing character assasination he also presented my telephonic conversation with one of my colleague in a complaint to cm window haryana how did he get hold of my telephonic call with some other person he hacked my mobile and emails and using them to as fake evidence to remove me from my job as a lady i am feeling helpless i filed a complaint on which a fir number was registered in camp palwal police station on even after more than months no investigation is done police is in collusion with accused and trying to save him they have not recorded my statement so far please take some tangible action and put the accused behind bars
23307,Online Financial Fraud,Internet Banking Related Fraud,fraudulent transaction happened by calling the mobile number provided in the sms stating that kseb electricity payment is due fraudster instructed to download quickpay app which resulted in loss of money suspecting merchant cashfreecom
21537,Online Financial Fraud,UPI Related Frauds,fraud through upi fraud phone pe bank bank of baroda including vijaya bank and dena bank account number total amount please hold reverse total amount in the complaint account and take necessary action
7828,Online Financial Fraud,Internet Banking Related Fraud,victim received message from suspect through whatsapp for reliance loan suspect contacted to victim and asked processing fees victim believed and paid money later than victim realized its scamamount lost rs suspect no
50145,Online and Social Media Related Crime,Cheating by Impersonation,an unknown person with mobile number is sending threatening messages to some of my contacts and asking my familyfriends to request me k

Language detection for stop word removal
(on hold)

In [ ]:
'''!pip install indic-nlp-library langdetect
!pip install polyglot
!pip install pyicu
!pip install pycld2

import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from indicnlp.tokenize import indic_tokenize
from indicnlp.normalize import indic_normalize
import re
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # Normalize Indic text
    if lang in ["hi", "bn", "ta", "te", "mr"]:
        normalizer = indic_normalize.Normalizer()
        text = normalizer.normalize(text)

    # Remove special characters and digits
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)

    # Tokenization
    words = text.split() if lang == "en" else indic_tokenize.trivial_tokenize(text)

    # Stopword removal and lemmatization
    if lang == "en":
        words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
    elif lang in ["hi", "bn", "ta", "te", "mr"]:
        words = [word for word in words if word not in indic_stopwords]

    return " ".join(words)

'''

'!pip install indic-nlp-library langdetect\n!pip install polyglot\n!pip install pyicu\n!pip install pycld2 \n\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom nltk.corpus import stopwords\nfrom nltk.stem import WordNetLemmatizer\nfrom indicnlp.tokenize import indic_tokenize\nfrom indicnlp.normalize import indic_normalize\nimport re\nimport nltk\n\nnltk.download(\'stopwords\')\nnltk.download(\'wordnet\')\nnltk.download(\'punkt\')\n\ndef preprocess_text(text):\n    if not isinstance(text, str):\n        return ""\n\n    # Normalize Indic text\n    if lang in ["hi", "bn", "ta", "te", "mr"]:\n        normalizer = indic_normalize.Normalizer()\n        text = normalizer.normalize(text)\n\n    # Remove special characters and digits\n    text = re.sub(r\'[^\\w\\s]\', \'\', text)\n    text = re.sub(r\'\\d+\', \'\', text)\n\n    # Tokenization\n    words = text.split() if lang == "en" else indic_tokenize.trivial_tokenize(text)\n\n    # Stopword removal and lemmatization\n    if lang =

## **Stop-word removal, lemmatization, vectorization, tokenization, normalization**

In [ ]:
! pip install nltk indic-nlp-library cleanlab unidecode autocorrect -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 14.1 MB/s eta 0:00:00


In [ ]:
import re
import nltk
import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))
indic_stopwords = set([
    "aur", "ki", "hai", "huyi", "ho", "mein", "ye", "ke", "jo", "saath", "ko",
    "bhi", "tatha", "par", "se", "kisi", "un", "apna", "tum", "main", "aap", "inhe",
    "in", "abhi", "ab", "woh", "hum", "unka", "is", "us", "kintu", "athva", "nahin",
    "kar","firto","fir","kese","esse","ka","kabhi","karna"
])
stop_words=stop_words.union(indic_stopwords)

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Normalize Unicode characters
    text = unidecode.unidecode(text)

    # Tokenization
    words = word_tokenize(text)

    # Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]

    # Removing stopwords
    words = [word for word in words if word not in stop_words]

    return ' '.join(words)

In [ ]:
# Apply preprocessing
df_train_cleaned["crimeaditionalinfo"] = df_train_cleaned.apply(lambda row: preprocess_text(row["crimeaditionalinfo"]), axis=1)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import cleanlab
from cleanlab.classification import CleanLearning
from sklearn.linear_model import LogisticRegression

In [ ]:
# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train_cleaned["crimeaditionalinfo"])

In [ ]:
'''
#Dont run unless you really have to check
tf_idf = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names_out())

final_df = tf_idf

print("{} rows".format(final_df.shape[0]))
final_df.T.nlargest(5, 0)
'''

'\n#Dont run unless you really have to check\ntf_idf = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names_out())\n\nfinal_df = tf_idf\n\nprint("{} rows".format(final_df.shape[0]))\nfinal_df.T.nlargest(5, 0)\n'

## Checking Mislabelled Data

AutoEncoder with LSTM Approach

In [ ]:
!pip install tensorflow -q

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Input, RepeatVector, TimeDistributed
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest


In [ ]:
df=df_train_cleaned.copy()

In [ ]:
# Load Pretrained GloVe Embeddings (100D)
embedding_index = {}
with open("/content/glove.6B.100d.txt", encoding="utf-8") as f:  # Replace with actual GloVe file
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coefs

In [ ]:
# Tokenization & Padding
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['crimeaditionalinfo'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df['crimeaditionalinfo'])
padded_sequences = pad_sequences(sequences, maxlen=200, padding='post')


In [ ]:
# Prepare Embedding Matrix
embedding_dim = 100  # Must match GloVe vector dimension
embedding_matrix = np.zeros((5000, embedding_dim))
for word, i in word_index.items():
    if i < 5000:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


In [ ]:
# Encode labels
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])
num_classes = len(label_encoder.classes_)


In [ ]:
# One-hot encode labels for multi-class classification
y_categorical = to_categorical(df['category_encoded'], num_classes=num_classes)

In [ ]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y_categorical, test_size=0.2, random_state=42)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import accuracy_score

class AccuracyCallback(Callback):
    def __init__(self, X_train, threshold_percentile=95):
        super(AccuracyCallback, self).__init__()
        self.X_train = X_train
        self.threshold_percentile = threshold_percentile

    def on_epoch_end(self, epoch, logs=None):
        X_train_pred = self.model.predict(self.X_train, verbose=0)

        # Compute Mean Squared Error per sample
        mse_per_sample = np.mean(np.power(self.X_train - X_train_pred, 2), axis=1)

        # Determine threshold for misclassification (top 5% MSE)
        threshold = np.percentile(mse_per_sample, self.threshold_percentile)

        # Binary classification: 1 = Correct, 0 = Mislabeled
        y_pred = (mse_per_sample < threshold).astype(int)  # Correct = 1, Mislabeled = 0
        y_true = np.ones_like(y_pred)  # All training samples are considered correct

        # Compute Accuracy
        accuracy = accuracy_score(y_true, y_pred)

        print(f"Epoch {epoch+1}: Accuracy = {accuracy:.4f}")

# Attach the callback during training
accuracy_callback = AccuracyCallback(X_train)

In [ ]:
# Define parameters
latent_dim = 128  # Latent space size
max_seq_length = 200  # Adjust according to your data
vocab_size = 5000  # Define based on your dataset

# Encoder
input_layer = Input(shape=(max_seq_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=100, trainable=True)(input_layer)
encoder = Bidirectional(LSTM(latent_dim, activation="tanh", return_sequences=False, dropout=0.2, recurrent_dropout=0.2))(embedding_layer)

# Decoder
decoder = RepeatVector(max_seq_length)(encoder)  # Repeat latent vector for each timestep
decoder = LSTM(256, activation="tanh", return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(decoder)
decoder = TimeDistributed(Dense(num_classes, activation="softmax"))(decoder)  # Softmax for multiclass

# Model
autoencoder = Model(input_layer, decoder)

In [ ]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0001, clipnorm=1.0)
autoencoder.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# Summary
autoencoder.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 200, 100)            │         500,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 256)                 │         234,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector (RepeatVector)         │ (None, 200, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 200, 256)            │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 200, 16)             │           4,112 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,263,920 (4.82 MB)

 Trainable params: 1,263,920 (4.82 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the Autoencoder
history = autoencoder.fit(X_train, np.repeat(y_train[:, np.newaxis, :], 200, axis=1),
                          epochs=5, validation_data=(X_test, np.repeat(y_test[:, np.newaxis, :], 200, axis=1)),
                          batch_size=32)

Epoch 1/5
2579/2579 ━━━━━━━━━━━━━━━━━━━━ 8419s 3s/step - accuracy: 0.6689 - loss: 1.1312 - val_accuracy: 0.7130 - val_loss: 0.8602
Epoch 2/5
2579/2579 ━━━━━━━━━━━━━━━━━━━━ 8549s 3s/step - accuracy: 0.7185 - loss: 0.8630 - val_accuracy: 0.7111 - val_loss: 0.8527
Epoch 3/5
2579/2579 ━━━━━━━━━━━━━━━━━━━━ 8479s 3s/step - accuracy: 0.7193 - loss: 0.8451 - val_accuracy: 0.7170 - val_loss: 0.8636
Epoch 4/5
2579/2579 ━━━━━━━━━━━━━━━━━━━━ 8458s 3s/step - accuracy: 0.7230 - loss: 0.8274 - val_accuracy: 0.7165 - val_loss: 0.8332
Epoch 5/5
2579/2579 ━━━━━━━━━━━━━━━━━━━━ 8602s 3s/step - accuracy: 0.7261 - loss: 0.8096 - val_accuracy: 0.7208 - val_loss: 0.8183


In [ ]:
# Extract encoded representations
encoder_model = Model(input_layer, encoder)
encoded_features = encoder_model.predict(X_test)  # Extract latent vectors for anomaly detection

645/645 ━━━━━━━━━━━━━━━━━━━━ 96s 142ms/step


In [ ]:
# Train Isolation Forest (Unsupervised Anomaly Detector)
iso_forest = IsolationForest(contamination=0.05, random_state=42)
iso_forest.fit(encoded_features)

IsolationForest(contamination=0.05, random_state=42)

In [ ]:
test_indices = df.index[X_test.shape[0]:]
df_test = df.iloc[test_indices].copy()
df_test['anomaly_flag'] = iso_forest.predict(encoded_features)
df_test['anomaly_flag'] = df_test['anomaly_flag'].map({1: 0, -1: 1})

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [ ]:
# Ground truth mislabeled cases
y_pred = autoencoder.predict(X_test)
y_pred_labels = np.argmax(y_pred[:, 0, :], axis=1)
df_test['predicted_category'] = label_encoder.inverse_transform(y_pred_labels)
df_test['mislabel_flag'] = df_test['category'] != df_test['predicted_category']

In [ ]:
# Save DataFrame as CSV
df_test.to_csv('relabelled_data_encoder.csv', index=False)

In [ ]:
# Compute Performance Metrics
accuracy = accuracy_score(df_test['mislabel_flag'], df_test['anomaly_flag'])
precision = precision_score(df_test['mislabel_flag'], df_test['anomaly_flag'])
recall = recall_score(df_test['mislabel_flag'], df_test['anomaly_flag'])
f1 = f1_score(df_test['mislabel_flag'], df_test['anomaly_flag'])

# Print Results
print(f"Number of mislabeled cases detected: {df_test['anomaly_flag'].sum()}")
print(f"Anomaly Detection Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")